In [11]:
!wget https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py

--2024-06-03 11:21:45--  https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1654 (1.6K) [text/plain]
Saving to: ‘average_precision.py.1’

average_precision.p 100%[===================>]   1.62K  --.-KB/s    in 0s      

2024-06-03 11:21:46 (67.3 MB/s) - ‘average_precision.py.1’ saved [1654/1654]



In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from average_precision import apk

In [2]:
import pyarrow
pyarrow.__version__

'10.0.1'

### Storage Optimization

- hex id to str
- categorization

In [9]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    # convert hex string to decimal equivalent
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)

In [3]:
%%time
import pandas as pd

transactions = pd.read_csv('data/transactions_train.csv', dtype={"article_id": "str"})
customers = pd.read_csv('data/customers.csv')
articles = pd.read_csv('data/articles.csv', dtype={"article_id": "str"})

CPU times: user 17.8 s, sys: 1.95 s, total: 19.7 s
Wall time: 19.7 s


In [4]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ GB


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [5]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id         3846387204
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [6]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 8.0 GB


In [7]:
transactions['customer_id'].nunique()

1362281

In [11]:
int(transactions["customer_id"][0][-16:], 16)

18439897732908966680

In [14]:
transactions['customer_id'] = customer_hex_id_to_int(transactions['customer_id'])
transactions['customer_id'].nunique()

1362281

In [18]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id          254306592
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [19]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       uint64 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 4.7 GB


In [20]:
transactions.t_dat = pd.to_datetime(transactions.t_dat, format='%Y-%m-%d')

In [25]:
print(transactions.t_dat.max())
transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

2020-09-22 00:00:00


In [27]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
 5   week              int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), uint64(1)
memory usage: 3.2 GB


In [29]:
transactions["article_id"][:5]

0    0663713001
1    0541518023
2    0505221004
3    0685687003
4    0685687004
Name: article_id, dtype: object

In [30]:
transactions.article_id = article_id_str_to_int(transactions.article_id)
articles.article_id = article_id_str_to_int(articles.article_id)

In [31]:
transactions.week = transactions.week.astype('int8')
transactions.sales_channel_id = transactions.sales_channel_id.astype('int8')
transactions.price = transactions.price.astype('float32')

In [32]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        int32         
 3   price             float32       
 4   sales_channel_id  int8          
 5   week              int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 788.2 MB


In [33]:
transactions.drop(columns='t_dat').info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   customer_id       uint64 
 1   article_id        int32  
 2   price             float32
 3   sales_channel_id  int8   
 4   week              int8   
dtypes: float32(1), int32(1), int8(2), uint64(1)
memory usage: 545.7 MB


**We have optimized the memory usage for the transactions dataframe**

**Now we are going to optimize the storage for customers and articles**

In [34]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355969 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 512.3 MB


In [35]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int32 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [36]:
customers['club_member_status'].unique()

array(['ACTIVE', nan, 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [37]:
customers.customer_id = customer_hex_id_to_int(customers.customer_id)

In [38]:
for col in ['FN', 'Active', 'age']:
    customers[col].fillna(-1, inplace=True)
    customers[col] = customers[col].astype('int8')

/tmp/ipykernel_27267/3619294266.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers[col].fillna(-1, inplace=True)
/tmp/ipykernel_27267/3619294266.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [39]:
customers.club_member_status = Categorize().fit_transform(customers[['club_member_status']]).club_member_status
customers.postal_code = Categorize().fit_transform(customers[['postal_code']]).postal_code
customers.fashion_news_frequency = Categorize().fit_transform(customers[['fashion_news_frequency']]).fashion_news_frequency

In [40]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   customer_id             1371980 non-null  uint64
 1   FN                      1371980 non-null  int8  
 2   Active                  1371980 non-null  int8  
 3   club_member_status      1371980 non-null  int8  
 4   fashion_news_frequency  1371980 non-null  int8  
 5   age                     1371980 non-null  int8  
 6   postal_code             1371980 non-null  int32 
dtypes: int32(1), int8(5), uint64(1)
memory usage: 22.2 MB


In [41]:
for col in articles.columns:
    if articles[col].dtype == 'object':
        articles[col] = Categorize().fit_transform(articles[[col]])[col]

In [42]:
articles.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   article_id                    105542 non-null  int32
 1   product_code                  105542 non-null  int64
 2   prod_name                     105542 non-null  int32
 3   product_type_no               105542 non-null  int64
 4   product_type_name             105542 non-null  int16
 5   product_group_name            105542 non-null  int8 
 6   graphical_appearance_no       105542 non-null  int64
 7   graphical_appearance_name     105542 non-null  int8 
 8   colour_group_code             105542 non-null  int64
 9   colour_group_name             105542 non-null  int8 
 10  perceived_colour_value_id     105542 non-null  int64
 11  perceived_colour_value_name   105542 non-null  int8 
 12  perceived_colour_master_id    105542 non-null  int64
 13  perceived_colo

In [43]:
for col in articles.columns:
    if articles[col].dtype == 'int64':
        articles[col] = articles[col].astype('int32')

In [44]:
transactions.sort_values(['t_dat', 'customer_id'], inplace=True)

In [46]:
%%time

transactions.to_parquet('data/transactions_train.parquet')
customers.to_parquet('data/customers.parquet')
articles.to_parquet('data/articles.parquet')

CPU times: user 1.68 s, sys: 876 ms, total: 2.55 s
Wall time: 2.55 s


## Development

In [3]:
transactions = pd.read_parquet('data/transactions_train.parquet')
customers = pd.read_parquet('data/customers.parquet')
articles = pd.read_parquet('data/articles.parquet')

In [4]:
# let's create a 5% sample of the entiriety of the data to speed up dev

sample = 0.05
customers_sample = customers.sample(frac=sample, replace=False)
customers_sample_ids = set(customers_sample['customer_id'])
transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
articles_sample_ids = set(transactions_sample["article_id"])
articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]

customers_sample.to_parquet(f'data/customers_sample_{sample}.parquet', index=False)
transactions_sample.to_parquet(f'data/transactions_train_sample_{sample}.parquet', index=False)
articles_sample.to_parquet(f'data/articles_train_sample_{sample}.parquet', index=False)

In [5]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600877 entries, 1702 to 31761187
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   t_dat             1600877 non-null  datetime64[ns]
 1   customer_id       1600877 non-null  uint64        
 2   article_id        1600877 non-null  int32         
 3   price             1600877 non-null  float32       
 4   sales_channel_id  1600877 non-null  int8          
 5   week              1600877 non-null  int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 51.9 MB


### Evaluation

In [6]:
from collections import defaultdict

val_week_purchases_by_cust = defaultdict(list)

val_week_purchases_by_cust.update(
    transactions[transactions.week == transactions.week.max()] \
        .groupby('customer_id')['article_id'] \
        .apply(list)
        .to_dict()
)

In [7]:
pd.to_pickle(dict(val_week_purchases_by_cust), 'data/val_week_purchases_by_cust.pkl')

In [10]:
sample_sub = pd.read_csv('data/sample_submission.csv')
valid_gt = customer_hex_id_to_int(sample_sub.customer_id) \
    .map(val_week_purchases_by_cust) \
    .apply(lambda xx: ' '.join('0' + str(x) for x in xx))

sample_sub.prediction = valid_gt
sample_sub.to_parquet('data/validation_ground_truth.parquet', index=False)

In [11]:
from average_precision import apk

def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

### Strong Starting Point

In [12]:
final_week = transactions[transactions.week == transactions.week.max()]
final_week.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week
index,,,,,,
31552433,2020-09-16,3177658828628418,869331006,0.030492,1,104
31552434,2020-09-16,3177658828628418,866731001,0.025407,1,104
31559847,2020-09-16,9802572176960585,896152002,0.033881,1,104
31559848,2020-09-16,9802572176960585,915526001,0.033881,1,104
31559686,2020-09-16,11484807043854349,878085001,0.025407,1,104


In [13]:
final_week['customer_id'].value_counts().describe()

count    68984.000000
mean         3.483576
std          3.535144
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        104.000000
Name: count, dtype: float64

In [14]:
bestsellers_last_week = set(transactions[transactions.week == transactions.week.max()].article_id.value_counts().index[:12])
bestsellers_week_ago = set(transactions[transactions.week == transactions.week.max()-1].article_id.value_counts().index[:12])


In [15]:
bestsellers_last_week

{448509014,
 714790020,
 751471001,
 762846027,
 866731001,
 909370001,
 915529003,
 915529005,
 918522001,
 923758001,
 924243001,
 924243002}

In [16]:
final_week.article_id.isin(bestsellers_last_week).mean(), final_week.article_id.isin(bestsellers_week_ago).mean()

(0.027967924897320556, 0.02349871624686344)

In [17]:
without_final_week = transactions[transactions.week != transactions.week.max()]

In [18]:
unique_bought_items = 0
last_purchase_repeated = 0
purchases_cust_with_no_history = 0
items_purchased_by_custs_with_no_history = []
week_of_earlier_purchase = []
i = 0

for c_id, df in final_week.groupby('customer_id'):
    purchases_final_week = set(df.article_id)
    unique_bought_items += len(purchases_final_week)
    
    purchase_history = without_final_week[without_final_week.customer_id == c_id]
    purchases_before = set(purchase_history[purchase_history.week == purchase_history.week.max()].article_id)
    week_of_earlier_purchase.append(purchase_history.week.max())
    
    if len(purchases_before) == 0:
        purchases_cust_with_no_history += len(purchases_final_week)
        items_purchased_by_custs_with_no_history += list(purchases_final_week)
    else:
        last_purchase_repeated += len(purchases_final_week.intersection(purchases_before))
    i += 1
    if i == 1000: break

In [19]:
wks_since_purchase = []
for week in week_of_earlier_purchase:
    if type(week) == np.int8:
        wks_since_purchase.append(104 - week)

In [20]:
# weeks elapsed between the purchase in the final week and earlier purchase
(pd.value_counts(wks_since_purchase)/len(wks_since_purchase)).head(20).cumsum()

/tmp/ipykernel_55729/2239400022.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  (pd.value_counts(wks_since_purchase)/len(wks_since_purchase)).head(20).cumsum()
/tmp/ipykernel_55729/2239400022.py:2: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  (pd.value_counts(wks_since_purchase)/len(wks_since_purchase)).head(20).cumsum()


1     0.196312
2     0.305857
3     0.396963
4     0.475054
5     0.539046
7     0.591106
6     0.634490
8     0.676790
13    0.708243
10    0.738612
11    0.767896
12    0.790672
9     0.812364
16    0.827549
15    0.838395
23    0.849241
14    0.860087
18    0.870933
21    0.880694
17    0.889371
Name: count, dtype: float64

In [21]:
unique_bought_items, last_purchase_repeated, purchases_cust_with_no_history

(3039, 80, 238)

In [22]:
np.mean([itm in bestsellers_last_week for itm in items_purchased_by_custs_with_no_history])

0.02100840336134454

There are not that many repeat purchases either. Though a vast majority of customers are repeat customers.

And new customers are not buying bestsellers all that much either.

I bet this could be improved if we did something useful with postal codes -- H&M operates across so many markets. The bestseller in one market doesn't have to be the bestseller in another.

A good model should outpeform this simple last purchase heuristic by a large margin. Still, let's implement it to be able to use down the road to refine our solution for situations where we don't have enough data / results are inconclusive.

In [23]:
last_three_weeks = without_final_week[without_final_week.week > without_final_week.week.max()-3]

In [24]:
best_sellers = last_three_weeks.groupby('week').apply(lambda df: df.value_counts('article_id').index[:12].tolist())

/tmp/ipykernel_55729/1825255527.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_sellers = last_three_weeks.groupby('week').apply(lambda df: df.value_counts('article_id').index[:12].tolist())


In [41]:
best_sellers

week
101    [898694001, 933706001, 751471001, 915526001, 9...
102    [915526001, 751471043, 751471001, 706016001, 9...
103    [909370001, 865799006, 918522001, 924243001, 4...
dtype: object

In [25]:
def purchase_history_to_preds(df):
    week_of_last_purchase = df.week.max()
    last_purchased_basket = df[df.week == week_of_last_purchase]
    purchased_items = last_purchased_basket.value_counts('article_id').index.tolist()
    purchased_items += best_sellers[last_purchased_basket.week.head(1).item()]
    return purchased_items[:12]

In [26]:
cust2preds2 = last_three_weeks.groupby(['customer_id']).apply(purchase_history_to_preds)

/tmp/ipykernel_55729/3793293915.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cust2preds2 = last_three_weeks.groupby(['customer_id']).apply(purchase_history_to_preds)


In [27]:
from dask.distributed import Client

client = Client(n_workers=24)
import dask.dataframe as dd

/home/niranjan/miniconda3/envs/fashion-rec-sys/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [28]:
ltw_dd = dd.from_pandas(last_three_weeks, npartitions=24)

In [29]:
ltw_dd

,t_dat,customer_id,article_id,price,sales_channel_id,week
npartitions=24,,,,,,
30744934,datetime64[ns],uint64,int32,float32,int8,int8
30778396,...,...,...,...,...,...
...,...,...,...,...,...,...
31514552,...,...,...,...,...,...
31548012,...,...,...,...,...,...


In [30]:
cust2preds = ltw_dd.groupby('customer_id').apply(purchase_history_to_preds, meta=('x', 'object')).compute()

/home/niranjan/miniconda3/envs/fashion-rec-sys/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 26.06 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/niranjan/miniconda3/envs/fashion-rec-sys/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/home/niranjan/miniconda3/envs/fashion-rec-sys/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warni

2024-06-03 13:53:08,315 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle db058d7ce46cea241e906b52e444b6bf initialized by task ('shuffle-transfer-db058d7ce46cea241e906b52e444b6bf', 12) executed on worker tcp://127.0.0.1:37051
2024-06-03 13:53:09,157 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle db058d7ce46cea241e906b52e444b6bf deactivated due to stimulus 'task-finished-1717402089.1408136'


In [31]:
client.close()

### Submission

In [32]:
last_week = last_three_weeks.week.max()
def get_preds_for_customer_id(c_id):
    if c_id in c_ids_with_predictions:
        pred_art_ids = cust2preds[c_id]
    else:
        pred_art_ids = best_sellers[last_week]
    return  ['0' + str(art_id) for art_id in pred_art_ids]

In [33]:
c_ids_with_predictions = set(cust2preds.keys())
preds = customer_hex_id_to_int(sample_sub.customer_id).map(get_preds_for_customer_id)

In [40]:
cust2preds[:2]

customer_id
28847241659200      [925246001, 915526001, 751471043, 751471001, 7...
1037449031262554    [857621001, 915526001, 751471043, 751471001, 7...
Name: x, dtype: object

In [34]:
sample_sub.prediction = preds
sample_sub.prediction = sample_sub.prediction.str.join(' ')

In [35]:
sub_name = 'bestsellers_single_week_logic'


In [37]:
sample_sub.to_csv(f'data/subs/{sub_name}.csv.gz', index=False)